## 유사도 분석

In [16]:
pip install --no-cache-dir Pororo

Note: you may need to restart the kernel to use updated packages.


In [2]:
import findspark

In [3]:
findspark.init()

In [4]:
import pyspark

In [5]:
from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import functools
import pandas as pd
import time

In [6]:
# books = '/home/hadoop/venv/book_infoo.csv'
books = 'hdfs://master:9000/user/practice/book_infoo.csv'

In [7]:
sc1 = SparkSession.builder.appName('book').getOrCreate()
df = sc1.read.option('multiline', True).csv(books, header=True, inferSchema=True)

In [8]:
new_df = df.orderBy(col('sub_category').asc())
new_df.toPandas()

,book_id,main_category,sub_category,book_title,book_author,publisher,publish_date,aladin_price,aladin_star,aladin_review,...,yes24_star,yes24_review,interpark_price,interpark_star,interpark_review,min price,avg stars,num of reviews,book_isbn,book_trailer
0,11154661,유아,0~3세,우리 아기 첫 오감발달 헝겊책,스텔라 배곳 글그림,어스본코리아,2016.08.25,13600,9.7,6,...,9.7,7,13600,9.3,3,13600,9.400,17,8809264180617,"우리 아기의 촉각, 시각, 청각 발달을 위한\r\n알록달록 헝겊책!\r\n\r\n알..."
1,2612684,유아,0~3세,사랑해 사랑해 사랑해,나뎃 로제티 슈스탁,보물창고,2006.12.20,9450,9.4,451,...,9.5,398,9450,9.5,338,9450,9.500,1265,8990794528,"아가야, 엄마 아빠는 너를 사랑한단다!\r\n\r\n'아기 그림책 보물창고' 시리즈..."
2,11802107,유아,0~3세,눈 코 입,백주희,보림,2017.03.10,9900,10.0,7,...,9.8,25,9900,9.7,5,9900,9.825,40,9788943310721,눈눈눈눈눈눈 코! 코코코코코코 입!\r\n\r\n\r\n\r\n표지를 보니 줄무늬 ...
3,6399291,유아,0~3세,뒹굴뒹굴 짝짝,연희,길벗어린이,2010.11.10,7650,9.6,16,...,9.4,19,7650,9.8,24,7650,9.700,61,9788955821154,"짝짝 뒹굴뒹굴 흔들흔들, 온몸을 움직이며 함께 놀아요!\r\n\r\n둥둥 아기그림책..."
4,2500218,유아,0~3세,사과가 쿵!,다다 히로시,보림,2009.06.20,9900,9.2,205,...,9.5,169,9900,9.3,186,9900,9.300,565,9788943306090,일본 태생의 다다 히로시의 『사과가 쿵!』. 1996년에 출간된 밀리언셀러 <사과가...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9257,16243574,과학/공학,화학,핵심일반화학,McMurry Fay Robinson,자유아카데미,2020.03.05,39000,0.0,0,...,10.0,3,39000,0.0,0,39000,4.850,3,9791158082499,"『핵심일반화학』은 일반화학을 다룬 이론서입니다. 〈화학적 도구〉, 〈원자, 분자 및..."
9258,6818627,과학/공학,화학,세상의 모든 원소 118,어도어 그레이,영림카디널,2012.01.31,28800,9.4,10,...,8.6,7,28800,9.3,3,28800,9.225,20,9788984011656,『세상의 모든 원소 118(특별판)』은 멘델레예프의 원소 주기율표에서 볼 수 있는 ...
9259,18030409,과학/공학,화학,처음부터 화학이 이렇게 쉬웠다면,사마키 다케오,한국경제신문,2021.02.26,13500,10.0,12,...,10.0,20,13500,10.0,3,13500,10.000,36,9788947546935,“이렇게 심플한 화학책은 처음이야!”\r\n\r\n과학 분야 50만 부 베스트셀러\...
9260,7037218,과학/공학,화학,화학 교과서는 살아있다,박태현 문상흡 하창식 이관영 오명숙 외,동아시아,2012.10.23,12600,10.0,5,...,8.3,3,12600,9.0,3,12600,9.175,11,9788962620580,우리의 일상생활 속에 숨어 있는 화학을 재미있게 공부하는 시간!\r\n\r\n화학을...


In [18]:
# -*- coding: utf-8 -*-
from pororo import Pororo
Pororo.available_tasks()

sts = Pororo(task='similarity', lang='ko')

ImportError: cannot import name 'Pororo'

In [ ]:
sub_book = new_df.groupBy('sub_category').agg(collect_list('book_title')).orderBy(col('sub_category').asc())
sub_id = new_df.groupBy('sub_category').agg(collect_list('book_id')).orderBy(col('sub_category').asc())
sub_book.toPandas()

In [ ]:
sub_id.toPandas()

In [ ]:
data = sub_book.join(sub_id, 'sub_category', 'left')
data = data.select('sub_category', col('collect_list(book_title)').alias('book_title'), col('collect_list(book_id)').alias('book_id'))
data.toPandas()

In [ ]:
rows = data.select('sub_category', 'book_title', 'book_id').collect()

In [ ]:

start = time.time()
for n in range(1):                       #두개의 dataframe을 union하기 위해 처음 데이터프레임을 생성하는 for loop
    book_title_recommend = []
    for i in range(len(rows[n][1])):     #처음 서브카테고리에서 모든 book_title의 횟수
        for j in range(len(rows[n][1])): #처음 서브카테고리에서 모든 book_title의 횟수
            try:
                if rows[n][1][i] == rows[n][1][j]:  #book_title이 같으면 통과
                    continue
                    
                res = sts(rows[n][1][i], rows[n][1][j])                                  #book_title끼리 유사도 비교
                book_title_recommend.append([rows[n][2][i], rows[n][2][j], res])         #리스트에 저장

            except Exception as e:
                continue
                
                
book_df = pd.DataFrame(book_title_recommend)
book_df.columns = ['book_id', 'book_recommend_id', 'score']
book_df = SparkSession.builder.getOrCreate().createDataFrame(book_df)                
book_df = book_df.orderBy(col('book_id').asc(), col('score').desc())

        
end = time.time()
print(end-start)

In [ ]:
rows[6]

In [ ]:
book_df.toPandas()

In [ ]:
'''카테고리 1번째부터 마지막카테고리까지 작업(0번째는 위에서 완료)'''
start = time.time()
for n in range(1, len(rows)):
    book_title_recommend = []
    for i in range(len(rows[n][1])):
        for j in range(len(rows[n][1])):
            try:
                if rows[n][1][i] == rows[n][1][j]:
                    continue
                res = sts(rows[n][1][i], rows[n][1][j])
                book_title_recommend.append([rows[n][2][i], rows[n][2][j], res])

            except Exception as e:
                continue
    
    book_all = pd.DataFrame(book_title_recommend)
    book_all.columns = ['book_id', 'book_recommend_id', 'score']
    book_all = SparkSession.builder.getOrCreate().createDataFrame(book_all)                
    book_all = book_all.orderBy(col('book_id').asc(), col('score').desc())
    book_df = book_df.union(book_all)
    print('category {} is done'.format(n))
    
end = time.time()
print(end-start)

In [ ]:
book_df.toPandas()

In [ ]:
book_df.coalesce(1).write.format("com.databricks.spark.csv"). \
                            option("header", "true").save("book_ex.csv")